## Adapatation of CMEMS data download provided by Favien
Seems far more proffesional and clean and also fixes my hardcod problem


In [1]:
import os
import copernicusmarine as cpm
from pathlib import Path
import ast
from datetime import datetime
import shutil

/Users/davidlewis/Desktop/Masters/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'satellite' directory exists inside 'data' folder
    satellite_dir = os.path.join(data_dir, 'satellite')
    if not os.path.exists(satellite_dir):
        os.makedirs(satellite_dir)
        print("'satellite' directory created inside 'data' folder.")
    else:
        print("'satellite' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()



'data' folder already exists in the parent directory.
'satellite' directory already exists inside 'data' folder.


In [3]:
datasets_to_extract = [
    "cmems_obs-sst_glo_phy_nrt_l4_P1D-m",  # daily L4 SST (near-real-time)
    "cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D"]          # bbp

# Per-dataset variable selection (single variable)
selected_variables = {
    "cmems_obs-sst_glo_phy_nrt_l4_P1D-m": ["analysed_sst"],
    "cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D": ["CHL"]
}

# Directory setup
parent_dir = os.path.abspath(os.path.join(os.getcwd()))
data_dir = "../data"
satellite_dir = "../data/satellite"

# Date and time setup
start_time = "2014-01-01T00:00:00"  # Format time as YYYY-MM-DD"T"HH:MM:SS
now = datetime.now()
end_time = now.strftime("2024-12-31T00:00:00")
end_time2 = now.strftime("20241231")
print(end_time2)

print(parent_dir)
print(data_dir)
print(satellite_dir)

20241231
/Users/davidlewis/Desktop/Masters/scripts
../data
../data/satellite


In [6]:
# Geographical bounds
min_depth = 0
max_depth = 10000
min_lon = -60
max_lon = -10
min_lat = 40
max_lat = 70

# Loop through each dataset
for dataset_to_extract in datasets_to_extract:
    # Dynamically set the output filename based on the dataset name
    saved_netCDF_as = f"CMEMS_{dataset_to_extract}_2021-2024.nc"
    filepath = os.path.join(satellite_dir, saved_netCDF_as)

    # Check if the file already exists
    if os.path.exists(filepath):
        response = input(f"Do you want to remove and overwrite the file \"{saved_netCDF_as}\" (y/n): ").strip().lower()
        if response == 'y':
            os.remove(filepath)
            print(f"{filepath} has been removed.")
            print(f"{saved_netCDF_as}. Proceeding to download.")
        else:
            print("Download canceled.")
            continue
    else:
        print(f"{saved_netCDF_as} does not exist. Proceeding to download.")
    
    # Perform the data subset operation
    cpm.subset(
        dataset_id=dataset_to_extract,
        variables=selected_variables.get(dataset_to_extract),
        minimum_longitude=min_lon,
        maximum_longitude=max_lon,
        minimum_latitude=min_lat,
        maximum_latitude=max_lat,
        start_datetime=start_time,
        end_datetime=end_time,
        minimum_depth=min_depth,
        maximum_depth=max_depth,
        output_filename=saved_netCDF_as,
        output_directory=satellite_dir
    )

../data/satellite/CMEMS_cmems_obs-sst_glo_phy_nrt_l4_P1D-m_2021-2024.nc has been removed.
CMEMS_cmems_obs-sst_glo_phy_nrt_l4_P1D-m_2021-2024.nc. Proceeding to download.


INFO - 2025-09-17T10:53:05Z - Selected dataset version: "202303"
INFO - 2025-09-17T10:53:05Z - Selected dataset part: "default"
WARNING - 2025-09-17T10:53:05Z - Some of your subset selection [2014-01-01 00:00:00+00:00, 2024-12-31 00:00:00+00:00] for the time dimension exceed the dataset coordinates [2021-01-01 00:00:00+00:00, 2025-09-16 00:00:00+00:00]
INFO - 2025-09-17T10:53:06Z - Starting download. Please wait...
100%|██████████| 323/323 [00:49<00:00,  6.53it/s]
INFO - 2025-09-17T10:53:56Z - Successfully downloaded to ../data/satellite/CMEMS_cmems_obs-sst_glo_phy_nrt_l4_P1D-m_2021-2024.nc


../data/satellite/CMEMS_cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_2021-2024.nc has been removed.
CMEMS_cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_2021-2024.nc. Proceeding to download.


INFO - 2025-09-17T10:54:54Z - Selected dataset version: "202311"
INFO - 2025-09-17T10:54:54Z - Selected dataset part: "default"
INFO - 2025-09-17T10:54:54Z - Starting download. Please wait...
100%|██████████| 1794/1794 [25:32<00:00,  1.17it/s]
INFO - 2025-09-17T11:20:27Z - Successfully downloaded to ../data/satellite/CMEMS_cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_2021-2024.nc


In [1]:
# --- Download ETOPO1 bathymetry subset into ../data ---
import pathlib, requests, xarray as xr

# Bounding box
min_lon, max_lon = -60, -10
min_lat, max_lat = 40, 70

# Convert to 0..360 for ERDDAP
to360 = lambda x: x % 360
lon0, lon1 = to360(min_lon), to360(max_lon)

# ERDDAP endpoint (ETOPO1 bedrock, lon 0–360, var=z)
erddap = "https://pae-paha.pacioos.hawaii.edu/erddap/griddap/etopo1_bedrock_lon360.nc"

# Build query (stride=1 → full 1-arcmin resolution)
query = f"?z[({min_lat}):1:({max_lat})][({lon0}):1:({lon1})]"
url = erddap + query
print("Downloading:", url)

# Output path
out_path = pathlib.Path("../data/etopo1_subset.nc")
out_path.parent.mkdir(parents=True, exist_ok=True)

# Download and save
r = requests.get(url, timeout=120)
r.raise_for_status()
out_path.write_bytes(r.content)
print("Saved bathymetry to", out_path)

# Optional: open with xarray to check
bathy = xr.open_dataset(out_path)
print(bathy)

Downloading: https://pae-paha.pacioos.hawaii.edu/erddap/griddap/etopo1_bedrock_lon360.nc?z[(40):1:(70)][(300):1:(350)]
Saved bathymetry to ../data/etopo1_subset.nc
<xarray.Dataset> Size: 22MB
Dimensions:    (latitude: 1801, longitude: 3001)
Coordinates:
  * latitude   (latitude) float64 14kB 40.0 40.02 40.03 ... 69.97 69.98 70.0
  * longitude  (longitude) float64 24kB 300.0 300.0 300.0 ... 350.0 350.0 350.0
Data variables:
    z          (latitude, longitude) float32 22MB ...
Attributes: (12/25)
    cdm_data_type:              Grid
    Conventions:                CF-1.4
    drawLandMask:               under
    Easternmost_Easting:        350.0
    geospatial_lat_max:         70.0
    geospatial_lat_min:         40.0
    ...                         ...
    Northernmost_Northing:      70.0
    sourceUrl:                  https://www.ngdc.noaa.gov/thredds/dodsC/globa...
    Southernmost_Northing:      40.0
    summary:                    ETOPO1 is a 1 arc-minute global relief model ...
 